In [1]:
import pandas as pd

df = pd.read_csv(r'gen_dataset.csv')
text_data = list(df["text"])
class_data = df["class"].to_numpy()
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1
class_data = list(class_data)
train_sarc_text_data = text_data[:int(len(text_data) * 0.8)]
test_sarc_text_data = text_data[int(len(text_data) * 0.8):]

train_sarc_class_data = class_data[:int(len(class_data) * 0.8)]
test_sarc_class_data = class_data[int(len(class_data) * 0.8):]
df

,class,id,text
0,0,1,"If that's true, then Freedom of Speech is doom..."
1,0,2,Neener neener - is it time to go in from the p...
2,0,3,"Just like the plastic gun fear, the armour pie..."
3,0,4,So geology is a religion because we weren't he...
4,0,5,Well done Monty. Mark that up as your first ev...
...,...,...,...
6515,1,6516,depends on when the baby bird died. run alon...
6516,1,6517,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,6518,so.. eh?? hows this sound? will it fly w...
6518,1,6519,"I think we should put to a vote, the right of ..."


In [5]:
import numpy as np
df = pd.read_csv(r'humor.csv')
text_data1 = list(df["text"])
class_data1 = list(df["humor"])
for i in range(len(class_data1)):
    if class_data1[i]==False:
        class_data1[i]=1
    else:
        class_data1[i]=0
# class_data1 = list(class_data1)    
train_humor_text_data = text_data1[:int(len(text_data1) * 0.8)]
test_humor_text_data = text_data1[int(len(text_data1) * 0.8):]

train_humor_class_data = class_data1[:int(len(class_data1) * 0.8)]
test_humor_class_data = class_data1[int(len(class_data1) * 0.8):]
# df
# test_irony_class_data[0]

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
from torchmetrics.classification import BinaryAccuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the pre-trained BERT model and tokenizer
model = AutoModel.from_pretrained('roberta-base')
model.to(device)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits



# Tokenize the text data
train_encodings = tokenizer(list(train_humor_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_humor_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_humor_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_humor_class_data))
)


# Define the training parameters
classifier = BertClassifier(model, num_classes=2)
classifier.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(3):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 80 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   

cuda


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch  1
---------------------------------------------------------------
Train Loss:  0.056121736660256286	Test Loss:  0.03847239568149671
Train Accuracy:  98.025625% 	Test Accuracy:  98.7125% 
Epoch  2
---------------------------------------------------------------
Train Loss:  0.028020883834641425	Test Loss:  0.04352116292400751
Train Accuracy:  99.03999999999999% 	Test Accuracy:  98.5% 
Epoch  3
---------------------------------------------------------------
Train Loss:  0.016207466096592543	Test Loss:  0.0411433466157876
Train Accuracy:  99.44125% 	Test Accuracy:  98.6125% 


In [7]:
# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]
train_encodings = tokenizer(list(train_sarc_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_sarc_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_sarc_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_sarc_class_data))
)


# Define the training parameters
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(10):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 40 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   
    print()

Epoch  1
-----
Train Loss:  0.7149636063473356	Test Loss:  0.6123995751869388
Train Accuracy:  55.329754601227% 	Test Accuracy:  65.9298780487805% 

Epoch  2
-----
Train Loss:  0.46281899868344967	Test Loss:  0.3923499395207661
Train Accuracy:  77.2430981595092% 	Test Accuracy:  81.73272362569483% 

Epoch  3
-----
Train Loss:  0.3423960055310302	Test Loss:  0.41797780191026085
Train Accuracy:  84.58588957055214% 	Test Accuracy:  79.34451219512195% 

Epoch  4
-----
Train Loss:  0.26220309521797247	Test Loss:  0.4035830003459279
Train Accuracy:  89.20628834355828% 	Test Accuracy:  81.65650411349971% 

Epoch  5
-----
Train Loss:  0.18420450601932462	Test Loss:  0.41084115497949647
Train Accuracy:  92.61886503067484% 	Test Accuracy:  83.38414634146342% 

Epoch  6
-----
Train Loss:  0.1221553559923556	Test Loss:  0.5785600315870308
Train Accuracy:  95.36042944785275% 	Test Accuracy:  82.16463414634147% 

Epoch  7
-----
Train Loss:  0.06778104719420198	Test Loss:  0.599125952982321
Train Acc

In [8]:
torch.save(classifier, "RoBERTaTransferHumor.pth")